In [1]:
import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [59]:
data = pd.read_csv("features.csv", index_col="match_id")

In [60]:
data.sample(5) # data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
25370,1439816864,1,17,5,1982,2028,23,1,0,10,...,4,2,1,-43.0,2127,0,1792,2047,48,63
110977,1449941096,1,19,4,1611,1508,25,0,0,12,...,1,3,1,3.0,1027,1,2047,1846,63,63
44642,1444171135,7,39,5,2206,1974,18,2,0,7,...,2,3,0,170.0,2302,0,0,2038,0,63
62887,1446988281,1,112,2,397,535,1,1,0,8,...,6,2,1,-44.0,4454,1,1796,512,63,0
91705,1449129491,0,68,2,397,499,0,0,1,10,...,3,2,0,-39.0,2631,0,0,1974,0,63


In [61]:
data = data.drop(['duration', 'tower_status_radiant',
 'tower_status_dire',
 'barracks_status_radiant',
 'barracks_status_dire', "start_time"], axis = 1)

In [62]:
list(data.columns)

['lobby_type',
 'r1_hero',
 'r1_level',
 'r1_xp',
 'r1_gold',
 'r1_lh',
 'r1_kills',
 'r1_deaths',
 'r1_items',
 'r2_hero',
 'r2_level',
 'r2_xp',
 'r2_gold',
 'r2_lh',
 'r2_kills',
 'r2_deaths',
 'r2_items',
 'r3_hero',
 'r3_level',
 'r3_xp',
 'r3_gold',
 'r3_lh',
 'r3_kills',
 'r3_deaths',
 'r3_items',
 'r4_hero',
 'r4_level',
 'r4_xp',
 'r4_gold',
 'r4_lh',
 'r4_kills',
 'r4_deaths',
 'r4_items',
 'r5_hero',
 'r5_level',
 'r5_xp',
 'r5_gold',
 'r5_lh',
 'r5_kills',
 'r5_deaths',
 'r5_items',
 'd1_hero',
 'd1_level',
 'd1_xp',
 'd1_gold',
 'd1_lh',
 'd1_kills',
 'd1_deaths',
 'd1_items',
 'd2_hero',
 'd2_level',
 'd2_xp',
 'd2_gold',
 'd2_lh',
 'd2_kills',
 'd2_deaths',
 'd2_items',
 'd3_hero',
 'd3_level',
 'd3_xp',
 'd3_gold',
 'd3_lh',
 'd3_kills',
 'd3_deaths',
 'd3_items',
 'd4_hero',
 'd4_level',
 'd4_xp',
 'd4_gold',
 'd4_lh',
 'd4_kills',
 'd4_deaths',
 'd4_items',
 'd5_hero',
 'd5_level',
 'd5_xp',
 'd5_gold',
 'd5_lh',
 'd5_kills',
 'd5_deaths',
 'd5_items',
 'first_blood_t

In [63]:
data = data.fillna(0)

In [64]:
X, y = data.drop('radiant_win', axis = 1), data["radiant_win"]

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [195]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y,
                                                   shuffle=True,
                                                   stratify=y)

In [67]:
model = RandomForestClassifier(n_estimators=100, n_jobs = 4)

In [68]:
%%time
model.fit(X_train, y_train)

Wall time: 31.5 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [69]:
ans = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, ans)

0.69126682474810275

In [70]:
np.mean(y_train)

0.5184992183428869

In [71]:
categorial = [
    'lobby_type',
    'first_blood_player1',
    'first_blood_player2',
] + list(filter(lambda x: "hero" in x, data.columns))
categorial

['lobby_type',
 'first_blood_player1',
 'first_blood_player2',
 'r1_hero',
 'r2_hero',
 'r3_hero',
 'r4_hero',
 'r5_hero',
 'd1_hero',
 'd2_hero',
 'd3_hero',
 'd4_hero',
 'd5_hero']

In [117]:
X_train_num = X_train.drop(categorial, axis = 1)
X_train_cat = X_train[categorial]

In [118]:
X_test_num = X_test.drop(categorial, axis = 1)
X_test_cat = X_test[categorial]

In [75]:
model = RandomForestClassifier(n_estimators=100, n_jobs = 4)

In [76]:
%%time
model.fit(X_train_num, y_train)

Wall time: 26.9 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
ans = model.predict_proba(X_test_num)[:, 1]
roc_auc_score(y_test, ans)

0.69143953679277681

In [78]:
from sklearn.linear_model import LogisticRegression

In [79]:
model = LogisticRegression(C=10)

In [80]:
%%time
model.fit(X_train_num, y_train)

Wall time: 36.5 s


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [81]:
ans = model.predict_proba(X_test_num)[:, 1]
roc_auc_score(y_test, ans)

0.71436169185839304

In [119]:
X_train_cat.sample(5)

,lobby_type,first_blood_player1,first_blood_player2,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
match_id,,,,,,,,,,,,,
109057,1,4.0,5.0,60,11,55,42,112,50,20,72,28,47
39395,7,0.0,0.0,40,69,22,85,12,36,57,8,26,71
78349,1,0.0,7.0,11,67,7,51,25,86,112,21,72,100
55050,0,8.0,4.0,39,71,57,112,28,86,73,11,7,97
40693,7,0.0,0.0,21,89,17,26,97,69,74,48,31,84


In [120]:
X_train_cat.lobby_type.unique()

array([7, 0, 1], dtype=int64)

In [121]:
for i in X_train_cat.lobby_type.unique():
    X_train_cat[f"type_{i}"] = X_train_cat["lobby_type"].apply(lambda x: x==i).astype(int)
    X_test_cat[f"type_{i}"] = X_test_cat["lobby_type"].apply(lambda x: x==i).astype(int)
X_train_cat = X_train_cat.drop("lobby_type", axis = 1)
X_test_cat = X_test_cat.drop("lobby_type", axis = 1)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [122]:
X_train_cat.sample(5)

,first_blood_player1,first_blood_player2,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero,type_7,type_0,type_1
match_id,,,,,,,,,,,,,,,
86499,1.0,5.0,50,39,7,59,72,11,8,87,55,30,0,0,1
86377,9.0,0.0,72,68,55,74,60,11,87,106,112,21,0,0,1
46411,0.0,0.0,28,51,50,7,11,68,103,60,86,69,0,0,1
20855,5.0,0.0,11,99,17,91,7,72,25,88,71,33,1,0,0
83441,0.0,0.0,71,8,50,90,39,11,30,85,47,112,0,0,1


In [123]:
from sklearn.preprocessing import OneHotEncoder

In [124]:
ohe = OneHotEncoder(sparse=False)
fbp_train = ohe.fit_transform(X_train_cat["first_blood_player1"].reshape((-1,1)))
fbp_test = ohe.transform(X_test_cat["first_blood_player1"].reshape((-1,1)))

fbp_train -= ohe.transform(X_train_cat["first_blood_player2"].reshape((-1,1)))
fbp_test -= ohe.transform(X_test_cat["first_blood_player2"].reshape((-1,1)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [125]:
X_train_cat = X_train_cat.reset_index()
df_fbp_train = pd.DataFrame(fbp_train)
df_fbp_train["match_id"] = X_train_cat["match_id"]

In [129]:
X_train_cat = pd.merge(X_train_cat, df_fbp_train, on="match_id")

In [130]:
X_train_cat = X_train_cat.drop(["first_blood_player2","first_blood_player1"], axis = 1)

In [132]:
X_test_cat = X_test_cat.reset_index()
df_fbp_test = pd.DataFrame(fbp_test)
df_fbp_test["match_id"] = X_test_cat["match_id"]

In [133]:
X_test_cat = pd.merge(X_test_cat, df_fbp_test, on="match_id")

In [134]:
X_test_cat = X_test_cat.drop(["first_blood_player2","first_blood_player1"], axis = 1)

In [139]:
heroes = pd.read_csv("heroes.csv")

In [153]:
heroes_ohe = OneHotEncoder(n_values=114, sparse=False)

In [179]:
heroes = heroes_ohe.transform(X_test_cat.r1_hero.reshape((-1,1)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [180]:
heroes = np.zeros_like(heroes)

for i in range(1,6):
    heroes += heroes_ohe.transform(X_test_cat[f'r{i}_hero'].reshape((-1,1)))
    heroes -= heroes_ohe.transform(X_test_cat[f'd{i}_hero'].reshape((-1,1)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [182]:
df_hero_test = pd.DataFrame(heroes)
df_hero_test["match_id"] = X_test_cat["match_id"]

In [183]:
X_test_cat = pd.merge(X_test_cat, df_hero_test, on="match_id")

In [184]:
X_test_cat = X_test_cat.drop(list(filter(lambda x: "hero" in x, data.columns)), axis = 1)

In [190]:
X_test_num = X_test_num.reset_index()
X_test = pd.merge(X_test_num, X_test_cat, on="match_id")

In [189]:
X_train = pd.merge(X_train_num, X_train_cat, on="match_id")

In [191]:
model = LogisticRegression(C=10)

In [192]:
model.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [193]:
ans = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, ans)

0.7354035070726932

In [194]:
import catboost

In [204]:
model = catboost.CatBoostClassifier(verbose=False, thread_count=4)

In [207]:
%%time
model.fit(X_train, y_train)

Wall time: 4min 29s


In [208]:
ans = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, ans)

0.74206188923738914